# 유튜브 조회수 기반 연관 단어 추출

이 코드에서는 유튜브 조회수를 기반으로 검색할 단어에 대해 조회수가 높은 연관 단어를 출력합니다.

먼저, 특정 단어가 포함된 데이터프레임을 필터링하고, 필터링된 데이터에서 각 원소별로 개수를 세어 내림차순으로 정렬합니다. 이후 각 단어에 대해 조회수를 합산하여 결과를 도출합니다.


In [7]:
import pandas as pd
import numpy as np
from collections import Counter

In [14]:
# 비디오 데이터 불러오기
# use_text 컬럼은 이전에 전처리를 끝낸 컬럼
df = pd.read_csv("../data/video_sample_20241013.csv", encoding="utf-8-sig", index_col=0)
df["use_text"] = df["use_text"].apply(lambda x: [word.strip() for word in x.split(",")])

In [41]:
# 특정 단어 정의
target_word = "먹방"

# 'use_text' 열에서 target_word가 포함된 행만 필터링
filtered_df = df[df['use_text'].apply(lambda x: target_word in x)]

# 필터링된 데이터에서 'use_text' 열의 리스트를 풀어서 모든 단어를 하나의 리스트로 만듭니다.
all_words = [word for words_list in filtered_df['use_text'] for word in words_list]

# 각 단어의 개수를 세어 Counter 객체 생성
word_count = Counter(all_words)

# target_word를 제외하고 나머지 단어들의 개수를 세어 딕셔너리 형태로 변환
sorted_word_count = {word: count for word, count in word_count.items() if word != target_word}

# 단어의 빈도를 기준으로 내림차순으로 정렬
sorted_word_count = dict(sorted(sorted_word_count.items(), key=lambda item: item[1], reverse=True))


In [43]:
# 결과를 저장할 딕셔너리 초기화
result = dict()

# sorted_word_count에서 각 단어와 빈도를 반복
for i, (key, value) in enumerate(sorted_word_count.items()):
    # 해당 단어가 포함된 행을 필터링하여 새로운 DataFrame 생성
    _df = filtered_df[filtered_df['use_text'].apply(lambda x: key in x)]
    
    # 필터링된 DataFrame에서 비디오 조회수를 합산
    video_views_sum = _df["video_views"].sum()
    
    # 조회수 합계를 정수로 변환
    score = int(video_views_sum)
    
    # score가 NaN이 아닐 경우에만 결과에 추가
    if not np.isnan(score):
        result[key] = score

# 조회수를 기준으로 내림차순 정렬
result = dict(sorted(result.items(), key=lambda x: x[1], reverse=True))

# 결과에서 상위 10개 항목만 선택
top_10_result = dict(list(result.items())[:10])

# 상위 10개 결과 출력
top_10_result


{'korea': 2609411,
 'life': 2585560,
 '차쥐뿔': 2491825,
 '지수': 2392946,
 '제니': 2381596,
 '마크': 2379054,
 'SUB': 2379041,
 'flower': 2378048,
 '이영지': 2377593,
 '도경수': 2377593}